# Projecto: Broker 

In [1]:
import math # Python maths 
import random # Random numbers for random actions
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader # Read and download the stock market data from the Yahoo website

from tqdm import tqdm_notebook, tqdm # View the percentage of completion of the task
from collections import deque # Expirience Replay (join past experiences )


Bad key "text.kerning_factor" on line 4 in
C:\Users\rafra\.conda\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
tf.__version__

'2.1.0'

### Paso 1: Construir la red neuronal del AI del Trader

In [4]:
class AI_Trader:
   
    def __init__(self, state_size, action_space = 3, model_name = "AITrader"): # Manten, Compra, Vende
        
        """ Parametros del modelo """
        self.state_size = state_size # state_size: tamaño de los estados del agente inteligente
        self.action_space = action_space # action_space: Número de accciones que va  a poder tomar
        self.memory = deque(maxlen=2000) # Memoria de 2000 experiencias 
        self.inventory = [] # Inventario para guardar divisas que tenemos
        self.model_name = model_name # Nombre del modelo...
        
        """ Parametros de la fase de aprendizaje """
        self.gamma = 0.95 # Factor gamma de la ecuación de Bellman
        self.epsilon = 1.0 # Cuando el modelo no entrenado el valor del epsilon es 100%
        self.epsilon_final = 0.01 # Al final del algoritmo solo el 1% de las veces tomara una decisión aleatoria
        self.epsilon_decay = 0.995 # Ratio de decreción 99.5%
        
        self.model = self.model_builder()
        
    def model_builder(self):
        
        model = Sequential()
        
        """ Capas Ocultas """
        model.add(Dense(units=32, activation='relu', input_dim= self.state_size)) # La capa de entrada espera 3 datos
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        
        """ Capa de Salida """
        model.add(Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer = Adam(lr=0.001)) # Mean Square Error función de perdida
        # Permite actualizar el velor de perdidas de la información NO de la acción 
        
        return model
    
    def trade(self, state):
        
        """ Acciones aleatorias en la fase de aprendizaje """
        if random.random() <= self.epsilon: # Generar una decisión aleatoria
            return random.randrange(self.action_space) # Acción aleatoria entre 0 y 2 inclusivo
        
        """ Evaluar estado actual y predecir acción"""
        actions = self.model.predict(state) # Si no llegamos al if definimos el conjunto de acciones que el modelo prediga
        return np.argmax(actions[0]) # ArgMax nos devuelve la acción mayor (número mayor)
    
    def batch_train(self, batch_size): # Entrenamiento por bloques batch_size : tamaño del bloque de entrenamiento deseado
        
        batch = []
        # Se recurre a la memoria en las ultimmas etapas ya que deque() encola los ultimos elementos en los ultimos indices
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i]) # Los ultimos fragmentos de memoria quedan almacenados
        
        # Cada uno de los objetos en el batch consta de un estado, una acción, de una recompenza, cual fue el siguiente estado
        # Y una variable done para si finalizó el ejercicio totalmente
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done: # si el agente no esta en el estado terminal si no termino con la fase de entrenamiento
                """ recompenza + gamma * el maximo de todos los posibles valores que tomaramos en el estado siguiente """
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])# Ecuación de Bellman
            
            target = self.model.predict(state) # Debo ir a donde me prediga el modelo
            target[0][action] = reward # para la acción actual que acaba de tomar el agente actualizo la recompenza
            
            self.model.fit(state, target, epochs=1, verbose=0) # backpropagation, 1 epoch para sacar la info de la actual eperiencia
            
        if self.epsilon > self.epsilon_final: # Decrementar el valor del epsilon
            self.epsilon *= self.epsilon_decay

### Paso 2: Pre-Procesado del dataset

In [5]:
def sigmoid(x): # función de activación utilmas etapas de la red neuronal
    return 1/ (1 + math.exp(-x))

In [6]:
def stocks_price_format(n): # Define el formato de precios
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [7]:
def dataset_loader(stock_name): # dada una divisa procede a la carga de los datos
    
    dataset = data_reader.DataReader(stock_name, data_source='yahoo')
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    
    return close   

In [8]:
# data -> dataset, timestep -> fecha de la deseada predicción, ventana -> dias previos a la fecha para realizar el estudio
def state_creator(data, timestep, window_size): 
    
    starting_id = timestep - window_size + 1 # día en el que comienza el analisis
    
    if starting_id >= 0: # Chequear que el dia sea positivo y tengamos dias para navegar
        windowed_data = data[starting_id:timestep+1] # Todos los datos en ese rango de fechas
    else: # Si el dia es negativo
        windowed_data = -starting_id * [data[0]] + list(data[0:timestep+1]) # Se repetira el primer dia hasta llenar la ventana
    
    state = [] # vector de estados
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[1+1] - windowed_data[i])) # 
    
    return np.array([state])

### Paso 3: Cargar el Dataset

In [9]:
# Cargar divisa del mercado
stock_name = 'AAPL'
data = dataset_loader(stock_name)

In [10]:
data.head()

Date
2015-08-27    112.919998
2015-08-28    113.290001
2015-08-31    112.760002
2015-09-01    107.720001
2015-09-02    112.339996
Name: Close, dtype: float64

### Paso 4: Entrenar la AI trader

In [11]:
# Configurar los hyper parametros
window_size = 10 # Ventana de dias pasados que va a evaluar los datos
episodes = 10 # Cuantas veces vamos a recorrer el entorno

batch_size = 32 # No se actualizaran los pesos de la red neuronal hasta que alla pasado por 32 muestras
data_samples = len(data) - 1 # No podemos utilizar el ultimo dato de todos 

In [12]:
# Definir modelo del AI trader
trader = AI_Trader(window_size)

In [13]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Paso 5: Fase de entrenamiento

In [14]:
for episode in range(1, episodes + 1):
    
    print('Episodes {}/{}'.format(episode, episodes)) # Muestra el estado de los episodios
    
    # Tomando los datos, desde el dia 0 y la vetana de trabajo para irme dias atrás
    state = state_creator(data, 0, window_size + 1)
    
    total_profit = 0 # Beneficio del bucle (este va incrementando)
    trader.inventory = [] # Inventario del trader (acciones que se van comprando y no se han vendido)
    
    # PUNTO CLAVE para cada episodio se come todo el data set.
    for t in tqdm(range(data_samples)): # n días - 1
        
        action = trader.trade(state) # Acción a tomar a cabo (al principio totalmete aleatorias)
        
        # A partir de los datos hasta el momento, calculamos el siguiente dia/timestamp
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0 # Recompenza
        
        if action == 1: # Compra
            trader.inventory.append(data[t]) # Almacena en inventario
            print('AI Trader buy: ', stocks_price_format(data[t])) # Log de la acción
            
        elif action == 2 and len(trader.inventory) > 0: # Vende si tenemos alguna acción
            buy_price = trader.inventory.pop(0) # Vendemos la ultima acción
            
            # La recompenza será el valor máximo entre el Data Set para el tiempo actual, 
            # para el día que estamos vendiendo la acción menos el precio, (siempre positivo)
            reward = max(data[t] - buy_price, 0) 
            total_profit += data[t] - buy_price # Nos dira si el beneficio es positivo o negativo

            print('AI trader sell: ', stocks_price_format(data[t]), 
                  ' Profit: ', stocks_price_format(data[t] - buy_price) ) # Log de la acción
            
        
        if t == data_samples - 1: # Si el timestamp es el ultimo (último día)
            done = True
        else:
            done = False
        
        """ Experience Replay """
        trader.memory.append((state, action, reward, next_state, done)) # Tuplas de 5 elementos
        
        state = next_state
        
        if done:
            print("########################################")
            print('BENEFICIO TOTAL: {}'.format(total_profit))
            print("########################################")
            
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
        
    if episode % 10 == 0:
        trader.model.save('ai_trader_{}.h5'.format(episode))

  0%|                                                                                         | 0/1257 [00:00<?, ?it/s]

Episodes 1/10
AI Trader buy:  $ 107.720001
AI Trader buy:  $ 110.370003
AI Trader buy:  $ 109.269997
AI Trader buy:  $ 112.309998
AI trader sell:  $ 112.570000  Profit:  $ 4.849998
AI trader sell:  $ 114.209999  Profit:  $ 3.839996
AI trader sell:  $ 116.279999  Profit:  $ 7.010002
AI Trader buy:  $ 113.919998
AI Trader buy:  $ 113.449997
AI trader sell:  $ 113.400002  Profit:  $ 1.090004
AI trader sell:  $ 114.320000  Profit:  $ 0.400002
AI Trader buy:  $ 112.440002
AI Trader buy:  $ 109.059998
AI Trader buy:  $ 110.300003
AI trader sell:  $ 109.580002  Profit:  - $ 3.869995
AI Trader buy:  $ 110.379997
AI trader sell:  $ 110.779999  Profit:  - $ 1.660004
AI Trader buy:  $ 112.120003
AI trader sell:  $ 111.599998  Profit:  $ 2.540001


  3%|██                                                                              | 33/1257 [00:04<02:56,  6.93it/s]

AI trader sell:  $ 110.209999  Profit:  - $ 0.090004


  3%|██▏                                                                             | 35/1257 [00:11<37:59,  1.87s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▉| 1255/1257 [1:17:05<00:15,  7.94s/it]

AI trader sell:  $ 497.480011  Profit:  $ 318.510010


100%|███████████████████████████████████████████████████████████████████████████▉| 1256/1257 [1:17:13<00:07,  7.97s/it]

AI trader sell:  $ 503.429993  Profit:  $ 325.199997
########################################
BENEFICIO TOTAL: 40461.97006225586
########################################


100%|████████████████████████████████████████████████████████████████████████████| 1257/1257 [1:17:21<00:00,  3.69s/it]
